In [395]:
import pandas as pd 
import numpy as np 
import json

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical



import matplotlib.pyplot as plt 
%matplotlib inline 

# Loading the Training and the Test Sets

In [396]:
train = pd.read_json('train/processed/train.json')
test = pd.read_json('test/processed/test.json')

In [397]:
train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [398]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1604 entries, 0 to 1603
Data columns (total 5 columns):
band_1        1604 non-null object
band_2        1604 non-null object
id            1604 non-null object
inc_angle     1604 non-null object
is_iceberg    1604 non-null int64
dtypes: int64(1), object(4)
memory usage: 75.2+ KB


In [399]:
train['inc_angle'].value_counts().head()

na         133
34.4721     23
42.5591     16
36.1061     15
33.6352     15
Name: inc_angle, dtype: int64

We right away notice that there are some NA valus in the inc_angle column 

In [400]:
train['inc_angle'].value_counts().head().index[0]

'na'

In [401]:
'''Replacing the NAs with 0s'''
train['inc_angle'] = train.inc_angle.replace(train['inc_angle'].value_counts().head().index[0], 0)

In [402]:
'''Validating the Replacements'''
train['inc_angle'].value_counts().head()

0.0000     133
34.4721     23
42.5591     16
33.6352     15
36.1061     15
Name: inc_angle, dtype: int64

In [403]:
train.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [404]:
train.is_iceberg.value_counts()

0    851
1    753
Name: is_iceberg, dtype: int64

In [405]:
'''Preprocessing the image to be fed into the model'''

b1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train["band_1"]])
b2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_2']])

b1 = b1/255.
b2 = b2/255.

In [406]:
X = np.stack([b1,b2], axis=3)
y = to_categorical(train['is_iceberg'])

In [407]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [408]:
X_train.shape

(1283, 75, 75, 2)

In [409]:
y_test.shape

(321, 2)

# Buidling a Simple Keras Model

In [410]:
'''Setting up the Model'''
model=Sequential()

'''Setting up the Layers'''
model.add(Conv2D(filters=15, kernel_size=(5,5), strides=(1,1), input_shape=(75,75,2), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=30, kernel_size=(4,4), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=45, kernel_size=(4,4), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=60, kernel_size=(4,4), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(50, activation='relu'))

'''Setting up the Output'''
model.add(Dense(2, activation='sigmoid'))

In [411]:
'''Compiling the Model'''
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [412]:
'''Fitting the model'''
model.fit(X_train,y_train, validation_data=[X_test, y_test], epochs=75, verbose=1)

Train on 1283 samples, validate on 321 samples
Epoch 1/75
1283/1283 [==============================] - 12s 9ms/step - loss: 0.6782 - acc: 0.5292 - val_loss: 0.6176 - val_acc: 0.5358
Epoch 2/75
1283/1283 [==============================] - 9s 7ms/step - loss: 0.5947 - acc: 0.6345 - val_loss: 0.5812 - val_acc: 0.6698
Epoch 3/75
1283/1283 [==============================] - 8s 6ms/step - loss: 0.5533 - acc: 0.6828 - val_loss: 0.5521 - val_acc: 0.6760
Epoch 4/75
1283/1283 [==============================] - 8s 6ms/step - loss: 0.5395 - acc: 0.6859 - val_loss: 0.5472 - val_acc: 0.6791
Epoch 5/75
1283/1283 [==============================] - 7s 6ms/step - loss: 0.5078 - acc: 0.7397 - val_loss: 0.4597 - val_acc: 0.8193
Epoch 6/75
1283/1283 [==============================] - 7s 6ms/step - loss: 0.4954 - acc: 0.7436 - val_loss: 0.5149 - val_acc: 0.7072
Epoch 7/75
1283/1283 [==============================] - 7s 6ms/step - loss: 0.4851 - acc: 0.7498 - val_loss: 0.4246 - val_acc: 0.8069
Epoch 8/75
128

1283/1283 [==============================] - 8s 6ms/step - loss: 0.0971 - acc: 0.9657 - val_loss: 0.3743 - val_acc: 0.8785
Epoch 62/75
1283/1283 [==============================] - 8s 6ms/step - loss: 0.0612 - acc: 0.9758 - val_loss: 0.4379 - val_acc: 0.8879
Epoch 63/75
1283/1283 [==============================] - 8s 6ms/step - loss: 0.0480 - acc: 0.9844 - val_loss: 0.4758 - val_acc: 0.8816
Epoch 64/75
1283/1283 [==============================] - 8s 6ms/step - loss: 0.0433 - acc: 0.9899 - val_loss: 0.4436 - val_acc: 0.8910
Epoch 65/75
1283/1283 [==============================] - 8s 6ms/step - loss: 0.0403 - acc: 0.9906 - val_loss: 0.4653 - val_acc: 0.8816
Epoch 66/75
1283/1283 [==============================] - 8s 6ms/step - loss: 0.0392 - acc: 0.9883 - val_loss: 0.4750 - val_acc: 0.8879
Epoch 67/75
1283/1283 [==============================] - 8s 6ms/step - loss: 0.0292 - acc: 0.9938 - val_loss: 0.4591 - val_acc: 0.8910
Epoch 68/75
1283/1283 [==============================] - 9s 7ms/ste

In [413]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_94 (Conv2D)           (None, 71, 71, 15)        765       
_________________________________________________________________
max_pooling2d_91 (MaxPooling (None, 35, 35, 15)        0         
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 32, 32, 30)        7230      
_________________________________________________________________
max_pooling2d_92 (MaxPooling (None, 16, 16, 30)        0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 13, 13, 45)        21645     
_________________________________________________________________
max_pooling2d_93 (MaxPooling (None, 6, 6, 45)          0         
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 3, 3, 60)          43260     
__________

# Getting Ready for some predictions 

In [414]:
'''Preprocessing the test image to be fed into the model'''
b1_test = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test["band_1"]])
b2_test = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test['band_2']])

b1_test = b1_test/255.
b2_test = b2_test/255.

In [415]:
X_test_submission = np.stack([b1_test,b2_test], axis=3)

In [416]:
pred = model.predict_classes(X_test_submission)

In [417]:
pred[0:10]

array([0, 0, 1, 1, 1, 0, 0, 1, 0, 0])

In [418]:
submit_df = pd.DataFrame({'id': test['id'], 'is_iceberg': pred})
submit_df.to_csv('naive_submission.csv', index=False)